In [1]:
"""
MODEL EVALUATION WITH MLFLOW - RESEARCH NOTEBOOK
================================================
Modern MLflow integration with secure credential management:
- Environment variable management with python-dotenv
- Secure DagHub authentication
- Comprehensive model evaluation metrics
- Proper MLflow tracking and logging
- Model registry integration
"""

import os
import sys
from pathlib import Path
import logging
from typing import Optional, Dict, Any
import warnings

# Suppress unnecessary warnings
warnings.filterwarnings('ignore')

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

In [2]:
%pwd

'c:\\Users\\asus\\Desktop\\Deep Learning project\\Chest-Cancer-Classification\\research'

In [3]:
# Navigate to project root
project_root = Path(__file__).resolve().parent.parent if '__file__' in globals() else Path.cwd().parent
os.chdir(project_root)
print(f"✓ Working directory: {os.getcwd()}")

✓ Working directory: c:\Users\asus\Desktop\Deep Learning project\Chest-Cancer-Classification


In [4]:
# Load environment variables for secure credential management
from dotenv import load_dotenv

env_path = Path('.env')
if env_path.exists():
    load_dotenv(env_path)
    print("✓ Environment variables loaded from .env")
    print("✓ Secure credential management enabled")
    
    # Verify required environment variables
    required_vars = ['DAGSHUB_REPO_OWNER', 'DAGSHUB_REPO_NAME', 'MLFLOW_TRACKING_URI']
    missing_vars = [var for var in required_vars if not os.getenv(var)]
    
    if missing_vars:
        print(f"⚠ Warning: Missing environment variables: {missing_vars}")
        print("  Update your .env file with these values")
    else:
        print("✓ All required credentials configured")
else:
    print("⚠ Warning: .env file not found")
    print("  Create .env from .env.example for secure credential management")
    print("  Falling back to hardcoded values (NOT RECOMMENDED for production)")

✓ Environment variables loaded from .env
✓ Secure credential management enabled
✓ All required credentials configured


In [5]:
# Initialize DagHub with credentials from environment variables
import dagshub

# Get credentials from environment (secure practice)
dagshub_owner = os.getenv('DAGSHUB_REPO_OWNER', 'CodeBy-HP')
dagshub_repo = os.getenv('DAGSHUB_REPO_NAME', 'chest-cancer-classification')

# Initialize DagHub integration
try:
    dagshub.init(
        repo_owner=dagshub_owner,
        repo_name=dagshub_repo,
        mlflow=True
    )
    print(f"✓ DagHub initialized: {dagshub_owner}/{dagshub_repo}")
    print("✓ MLflow tracking enabled")
except Exception as e:
    print(f"⚠ DagHub initialization warning: {e}")
    print("  MLflow will use local tracking")

2025-12-13 00:27:47,659 - httpx - INFO - HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"


Accessing as CodeBy-HP

2025-12-13 00:27:47,700 - dagshub - INFO - Accessing as CodeBy-HP
2025-12-13 00:27:49,819 - httpx - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/CodeBy-HP/chest-cancer-classification "HTTP/1.1 200 OK"
2025-12-13 00:27:50,987 - httpx - INFO - HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"


Initialized MLflow to track repo "CodeBy-HP/chest-cancer-classification"

2025-12-13 00:27:51,002 - dagshub - INFO - Initialized MLflow to track repo "CodeBy-HP/chest-cancer-classification"


Repository CodeBy-HP/chest-cancer-classification initialized!

2025-12-13 00:27:51,012 - dagshub - INFO - Repository CodeBy-HP/chest-cancer-classification initialized!


✓ DagHub initialized: CodeBy-HP/chest-cancer-classification
✓ MLflow tracking enabled


In [6]:
import tensorflow as tf
import numpy as np

print(f"TensorFlow version: {tf.__version__}")

# Verify model file exists
model_path = Path("artifacts/training/model.keras")
if model_path.exists():
    print(f"✓ Model file found: {model_path}")
else:
    print(f"❌ Model file not found: {model_path}")
    print("  Run training notebook first")

TensorFlow version: 2.20.0
✓ Model file found: artifacts\training\model.keras


In [7]:
# Load trained model for quick testing
try:
    model = tf.keras.models.load_model("artifacts/training/model.keras")
    print(f"✓ Model loaded successfully")
    print(f"  Model: {model.name}")
    print(f"  Parameters: {model.count_params():,}")
except Exception as e:
    print(f"❌ Failed to load model: {e}")

✓ Model loaded successfully
  Model: EfficientNetB0_ChestCancer
  Parameters: 4,057,253


In [8]:
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Any, List


@dataclass(frozen=True)
class EvaluationConfig:
    """
    Configuration for model evaluation.
    
    Modern practices:
    - Immutable configuration
    - Type hints
    - Validation
    """
    path_of_model: Path
    training_data: Path
    all_params: Dict[str, Any]
    mlflow_uri: str
    params_image_size: List[int]
    params_batch_size: int
    
    def __post_init__(self):
        """Validate configuration"""
        if not self.path_of_model.exists():
            raise FileNotFoundError(f"Model not found: {self.path_of_model}")
        if not self.training_data.exists():
            raise FileNotFoundError(f"Training data not found: {self.training_data}")

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories, save_json


class ConfigurationManager:
    """Modern configuration manager"""
    
    def __init__(
        self, 
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH
    ):
        """Initialize configuration"""
        try:
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            
            create_directories([self.config.artifacts_root])
            logging.info("✓ Configuration loaded successfully")
            
        except Exception as e:
            logging.error(f"Failed to load configuration: {e}")
            raise

    def get_evaluation_config(self) -> EvaluationConfig:
        """Get evaluation configuration with environment variable overrides"""
        
        # Get MLflow URI from environment (secure practice)
        mlflow_uri = os.getenv(
            'MLFLOW_TRACKING_URI',
            f"https://dagshub.com/{os.getenv('DAGSHUB_REPO_OWNER', 'CodeBy-HP')}/"
            f"{os.getenv('DAGSHUB_REPO_NAME', 'chest-cancer-classification')}.mlflow"
        )
        
        eval_config = EvaluationConfig(
            path_of_model=Path("artifacts/training/model.keras"),
            training_data=Path("artifacts/data_ingestion/Chest-CT-Scan-data"),
            mlflow_uri=mlflow_uri,
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        
        logging.info("✓ Evaluation config created")
        logging.info(f"  MLflow URI: {mlflow_uri}")
        
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import dagshub
import logging

print(f"MLflow version: {mlflow.__version__}")

MLflow version: 3.7.0


In [16]:
class Evaluation:
    """
    Modern model evaluation with MLflow tracking.
    
    Best practices:
    - Secure credential management from environment
    - Comprehensive metrics tracking
    - Model registry integration
    - Proper error handling
    - Production-ready logging
    """
    
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.logger = logging.getLogger(self.__class__.__name__)
        self.model = None
        self.valid_generator = None
        self.score = None

    def _valid_generator(self) -> tf.data.Dataset:
        """
        Create validation dataset using modern tf.data API.
        
        Returns:
            tf.data.Dataset: Validation dataset
        """
        try:
            image_size = tuple(self.config.params_image_size[:-1])
            batch_size = self.config.params_batch_size
            
            self.logger.info(f"Loading validation data from: {self.config.training_data}")
            
            # Create validation dataset (30% for thorough evaluation)
            self.valid_generator = tf.keras.utils.image_dataset_from_directory(
                directory=str(self.config.training_data),
                validation_split=0.30,
                subset="validation",
                seed=123,
                image_size=image_size,
                batch_size=batch_size,
                shuffle=False,  # Don't shuffle for consistent evaluation
                label_mode='categorical'
            )
            
            # Get class names BEFORE transformations (important!)
            class_names = self.valid_generator.class_names
            self.logger.info(f"✓ Classes detected: {class_names}")
            
            # Normalize pixel values
            normalization_layer = tf.keras.layers.Rescaling(1./255)
            self.valid_generator = self.valid_generator.map(
                lambda x, y: (normalization_layer(x), y),
                num_parallel_calls=tf.data.AUTOTUNE
            )
            
            # Optimize performance
            self.valid_generator = self.valid_generator.cache().prefetch(
                buffer_size=tf.data.AUTOTUNE
            )
            
            self.logger.info(f"✓ Validation data loaded and preprocessed")
            
            return self.valid_generator
            
        except Exception as e:
            self.logger.error(f"Failed to create validation generator: {e}")
            raise

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        """
        Load model from .keras file.
        
        Args:
            path: Path to model file
            
        Returns:
            tf.keras.Model: Loaded model
        """
        try:
            if not path.exists():
                raise FileNotFoundError(f"Model file not found: {path}")
            
            model = tf.keras.models.load_model(path)
            logging.info(f"✓ Model loaded from: {path}")
            
            return model
            
        except Exception as e:
            logging.error(f"Failed to load model: {e}")
            raise

    def evaluation(self) -> Dict[str, float]:
        """
        Evaluate model and return metrics.
        
        Returns:
            Dict: Evaluation metrics
        """
        try:
            self.logger.info("Starting model evaluation...")
            
            # Load model
            self.model = self.load_model(self.config.path_of_model)
            
            # Create validation data
            self._valid_generator()
            
            # Evaluate model
            self.logger.info("Evaluating model on validation data...")
            results = self.model.evaluate(self.valid_generator, verbose=1)
            
            # Extract metrics (based on model compilation)
            metric_names = self.model.metrics_names
            self.score = dict(zip(metric_names, results))
            
            self.logger.info("✓ Evaluation completed")
            self.logger.info(f"  Metrics: {self.score}")
            
            # Save scores locally
            self.save_score()
            
            return self.score
            
        except Exception as e:
            self.logger.error(f"Evaluation failed: {e}")
            raise

    def save_score(self) -> None:
        """Save evaluation scores to JSON file"""
        try:
            save_json(path=Path("scores.json"), data=self.score)
            self.logger.info("✓ Scores saved to scores.json")
        except Exception as e:
            self.logger.error(f"Failed to save scores: {e}")
            raise

    def log_into_mlflow(self) -> None:
        """
        Log experiment to MLflow with secure credential management.
        
        Modern best practices:
        - Credentials from environment variables
        - Comprehensive parameter and metric logging
        - Model registry integration
        - Proper error handling
        """
        try:
            # Set MLflow tracking URI from config
            mlflow.set_tracking_uri(self.config.mlflow_uri)
            self.logger.info(f"✓ MLflow tracking URI set: {self.config.mlflow_uri}")
            
            # Get tracking URL type
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
            
            # Start MLflow run
            with mlflow.start_run():
                self.logger.info("✓ MLflow run started")
                
                # Log all parameters from params.yaml
                self.logger.info("Logging parameters...")
                mlflow.log_params(self.config.all_params)
                
                # Log all evaluation metrics
                self.logger.info("Logging metrics...")
                mlflow.log_metrics(self.score)
                
                # Log model to MLflow
                self.logger.info("Logging model...")
                
                # Model registry works with remote tracking (DagHub)
                if tracking_url_type_store != "file":
                    self.logger.info("Remote tracking detected - registering model...")
                    
                    # Register model with versioning
                    # Note: MLflow automatically handles .keras format in TF 3.0+
                    mlflow.keras.log_model(
                        self.model,
                        artifact_path="model",
                        registered_model_name="EfficientNetB0_ChestCancer"
                    )
                    
                    self.logger.info("✓ Model registered in MLflow Model Registry")
                else:
                    # Local tracking (file store)
                    self.logger.info("Local tracking detected - logging model...")
                    
                    mlflow.keras.log_model(
                        self.model,
                        artifact_path="model"
                    )
                    
                    self.logger.info("✓ Model logged to MLflow (local)")
                
                # Get run ID for reference
                run_id = mlflow.active_run().info.run_id
                self.logger.info(f"✓ MLflow run completed: {run_id}")
                
                print("\n" + "="*60)
                print("✓ MLFLOW LOGGING SUCCESSFUL")
                print("="*60)
                print(f"Run ID: {run_id}")
                print(f"Tracking URI: {self.config.mlflow_uri}")
                print("="*60 + "\n")
                
        except Exception as e:
            self.logger.error(f"MLflow logging failed: {e}")
            self.logger.error("Check your DagHub credentials in .env file")
            raise

In [17]:
# MAIN EXECUTION PIPELINE
# Production-ready evaluation with MLflow tracking

if __name__ == "__main__":
    try:
        print("\n" + "="*60)
        print("MODEL EVALUATION & MLFLOW TRACKING")
        print("="*60 + "\n")
        
        # Initialize configuration
        config_manager = ConfigurationManager()
        eval_config = config_manager.get_evaluation_config()
        
        # Initialize evaluation
        evaluation = Evaluation(eval_config)
        
        # Step 1: Evaluate model
        print("Step 1/3: Evaluating model...")
        metrics = evaluation.evaluation()
        
        # Display results
        print("\n" + "-"*60)
        print("EVALUATION RESULTS")
        print("-"*60)
        for metric_name, value in metrics.items():
            print(f"{metric_name:.<40} {value:.4f}")
        print("-"*60 + "\n")
        
        # Step 2: Save scores
        print("Step 2/3: Saving scores...")
        evaluation.save_score()
        print("✓ Scores saved to scores.json")
        
        # Step 3: Log to MLflow
        print("\nStep 3/3: Logging to MLflow...")
        evaluation.log_into_mlflow()
        
        print("\n" + "="*60)
        print("✓ EVALUATION COMPLETED SUCCESSFULLY")
        print("="*60 + "\n")
        print("📊 View results:")
        print(f"  - Local: scores.json")
        print(f"  - MLflow UI: {eval_config.mlflow_uri}")
        print("\n✨ All done!")
        
    except FileNotFoundError as e:
        print(f"\n❌ FILE ERROR: {e}")
        print("   Ensure training completed successfully")
    except Exception as e:
        print(f"\n❌ ERROR: {e}")
        print("\nTroubleshooting:")
        print("1. Check .env file has correct DagHub credentials")
        print("2. Ensure model training completed successfully")
        print("3. Verify internet connection for DagHub/MLflow")
        import traceback
        traceback.print_exc()
        raise

2025-12-13 00:32:37,408 - cnnClassifierLogger - INFO - yaml file: config\config.yaml loaded successfully
2025-12-13 00:32:37,419 - cnnClassifierLogger - INFO - yaml file: params.yaml loaded successfully
2025-12-13 00:32:37,425 - cnnClassifierLogger - INFO - created directory at: artifacts
2025-12-13 00:32:37,427 - root - INFO - ✓ Configuration loaded successfully
2025-12-13 00:32:37,431 - root - INFO - ✓ Evaluation config created
2025-12-13 00:32:37,434 - root - INFO -   MLflow URI: https://dagshub.com/CodeBy-HP/chest-cancer-classification.mlflow
2025-12-13 00:32:37,436 - Evaluation - INFO - Starting model evaluation...



MODEL EVALUATION & MLFLOW TRACKING

Step 1/3: Evaluating model...


2025-12-13 00:32:39,444 - root - INFO - ✓ Model loaded from: artifacts\training\model.keras
2025-12-13 00:32:39,445 - Evaluation - INFO - Loading validation data from: artifacts\data_ingestion\Chest-CT-Scan-data


Found 466 files belonging to 2 classes.
Using 139 files for validation.


2025-12-13 00:32:39,516 - Evaluation - INFO - ✓ Classes detected: ['adenocarcinoma', 'normal']
2025-12-13 00:32:39,543 - Evaluation - INFO - ✓ Validation data loaded and preprocessed
2025-12-13 00:32:39,545 - Evaluation - INFO - Evaluating model on validation data...


18/18 ━━━━━━━━━━━━━━━━━━━━ 13s 394ms/step - accuracy: 1.0000 - auc: 1.0000 - loss: 0.5527 - precision: 1.0000 - recall: 1.0000


2025-12-13 00:32:52,135 - Evaluation - INFO - ✓ Evaluation completed
2025-12-13 00:32:52,136 - Evaluation - INFO -   Metrics: {'loss': 0.5526924729347229, 'compile_metrics': 1.0}
2025-12-13 00:32:52,138 - cnnClassifierLogger - INFO - json file saved at: scores.json
2025-12-13 00:32:52,139 - Evaluation - INFO - ✓ Scores saved to scores.json
2025-12-13 00:32:52,144 - cnnClassifierLogger - INFO - json file saved at: scores.json
2025-12-13 00:32:52,146 - Evaluation - INFO - ✓ Scores saved to scores.json
2025-12-13 00:32:52,149 - Evaluation - INFO - ✓ MLflow tracking URI set: https://dagshub.com/CodeBy-HP/chest-cancer-classification.mlflow



------------------------------------------------------------
EVALUATION RESULTS
------------------------------------------------------------
loss.................................... 0.5527
compile_metrics......................... 1.0000
------------------------------------------------------------

Step 2/3: Saving scores...
✓ Scores saved to scores.json

Step 3/3: Logging to MLflow...


2025-12-13 00:32:53,633 - Evaluation - INFO - ✓ MLflow run started
2025-12-13 00:32:53,636 - Evaluation - INFO - Logging parameters...
2025-12-13 00:32:54,141 - Evaluation - INFO - Logging metrics...
2025-12-13 00:32:54,608 - Evaluation - INFO - Logging model...
2025-12-13 00:32:54,610 - Evaluation - INFO - Remote tracking detected - registering model...
2025/12/13 00:32:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/13 00:32:57 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
Successfully registered model 'EfficientNetB0_ChestCancer'.
2025/12/13 00:34:07 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: EfficientNetB0_ChestCancer, version 1
Created version '1' of model 'EfficientNetB0_ChestCancer'.
2025-12-13 00:34:09,317 - Evaluation - INFO - ✓ Model registered in MLflow Model Registry
2025-12-13 00:34:09,319 - Evalua


✓ MLFLOW LOGGING SUCCESSFUL
Run ID: 120ba9f82978450f9a07632704569ede
Tracking URI: https://dagshub.com/CodeBy-HP/chest-cancer-classification.mlflow

🏃 View run burly-bug-834 at: https://dagshub.com/CodeBy-HP/chest-cancer-classification.mlflow/#/experiments/0/runs/120ba9f82978450f9a07632704569ede
🧪 View experiment at: https://dagshub.com/CodeBy-HP/chest-cancer-classification.mlflow/#/experiments/0

✓ EVALUATION COMPLETED SUCCESSFULLY

📊 View results:
  - Local: scores.json
  - MLflow UI: https://dagshub.com/CodeBy-HP/chest-cancer-classification.mlflow

✨ All done!
